In [3]:
import pandas as pd
import sqlite3

In [4]:
DATA_FOLDER = "../sources/data/"
db_path = DATA_FOLDER + "challenge.db"
sql_file_path = DATA_FOLDER + "challenge_db_create.sql" 
conn = sqlite3.connect(db_path)

In [5]:
session_sources_df = pd.read_sql_query("SELECT * FROM session_sources", conn)
session_costs_df = pd.read_sql_query("SELECT * FROM session_costs", conn)
attribution_customer_journey_df = pd.read_sql_query("SELECT * FROM attribution_customer_journey", conn)
conversions_df = pd.read_sql_query("SELECT * FROM conversions", conn)
attribution_customer_journey_df=attribution_customer_journey_df.rename(columns={"conversion_id": "conv_id"})


In [34]:
# Fill NaN values in 'cost' with 0
session_costs_df['cost'] = session_costs_df['cost'].fillna(0)

# Merge the two DataFrames on 'session_id'
merged_data = pd.merge(session_sources_df, session_costs_df, on='session_id')

# Group by 'channel_name' and 'event_date' and sum the costs
channel_reporting_cost = merged_data.groupby(['channel_name', 'event_date'])['cost'].sum().reset_index()

# Rename columns to match the desired output
channel_reporting_cost = channel_reporting_cost.rename(columns={'event_date': 'date'})

# Print the resulting DataFrame
print(channel_reporting_cost)

                 channel_name        date     cost
0    Affiliate & Partnerships  2023-08-29  525.360
1    Affiliate & Partnerships  2023-08-30  574.644
2    Affiliate & Partnerships  2023-08-31  572.769
3    Affiliate & Partnerships  2023-09-01  413.205
4    Affiliate & Partnerships  2023-09-02  395.707
..                        ...         ...      ...
145               YouTube Ads  2023-09-02    0.000
146               YouTube Ads  2023-09-03    0.000
147               YouTube Ads  2023-09-05    0.000
148               YouTube Ads  2023-09-06    0.000
149               YouTube Ads  2023-09-07    0.000

[150 rows x 3 columns]


In [10]:
# Merge attribution_customer_journey and session_sources on 'session_id'
merged_data = pd.merge(attribution_customer_journey_df, session_sources_df, on='session_id')

# Group by 'channel_name' and 'event_date' and sum the 'ihc' values
channel_reporting_ihc = merged_data.groupby(['channel_name', 'event_date'])['ihc'].sum().reset_index()

# Rename 'event_date' to 'date'
channel_reporting_ihc = channel_reporting_ihc.rename(columns={'event_date': 'date'})

# Print the resulting DataFrame
print(channel_reporting_ihc)

                 channel_name        date      ihc
0    Affiliate & Partnerships  2023-08-29   0.0000
1    Affiliate & Partnerships  2023-08-30   0.0000
2    Affiliate & Partnerships  2023-08-31   0.4969
3    Affiliate & Partnerships  2023-09-01  13.2897
4    Affiliate & Partnerships  2023-09-02   6.1631
..                        ...         ...      ...
112     Untracked Conversions  2023-09-03   9.0000
113     Untracked Conversions  2023-09-04  12.0000
114     Untracked Conversions  2023-09-05   3.0000
115     Untracked Conversions  2023-09-06   6.0000
116     Untracked Conversions  2023-09-07  10.0000

[117 rows x 3 columns]


In [32]:
# Merge conversions and attribution_customer_journey on 'conv_id'
merged_conversions = pd.merge(conversions_df, attribution_customer_journey_df, on='conv_id')

# Merge the result with session_sources on 'session_id'
merged_data = pd.merge(merged_conversions, session_sources_df, on='session_id')

# Calculate 'ihc_revenue' by multiplying 'ihc' and 'revenue'
merged_data['ihc_revenue'] = merged_data['ihc'] * merged_data['revenue']

# Group by 'channel_name' and 'event_date' and sum the 'ihc_revenue'
channel_reporting_ihc_revenue = merged_data.groupby(['channel_name', 'event_date'])['ihc_revenue'].sum().reset_index()

# Rename 'event_date' to 'date'
channel_reporting_ihc_revenue = channel_reporting_ihc_revenue.rename(columns={'event_date': 'date'})

# Print the resulting DataFrame
print(channel_reporting_ihc_revenue)

                 channel_name        date  ihc_revenue
0    Affiliate & Partnerships  2023-08-29     0.000000
1    Affiliate & Partnerships  2023-08-30     0.000000
2    Affiliate & Partnerships  2023-08-31    20.084698
3    Affiliate & Partnerships  2023-09-01   630.190266
4    Affiliate & Partnerships  2023-09-02   304.572345
..                        ...         ...          ...
112     Untracked Conversions  2023-09-03   497.900000
113     Untracked Conversions  2023-09-04   645.550000
114     Untracked Conversions  2023-09-05   107.660000
115     Untracked Conversions  2023-09-06   196.980000
116     Untracked Conversions  2023-09-07   818.150000

[117 rows x 3 columns]


In [6]:
# --- Calculate channel_reporting_cost ---
session_costs_df['cost'] = session_costs_df['cost'].fillna(0)
merged_data_cost = pd.merge(session_sources_df, session_costs_df, on='session_id')
channel_reporting_cost = merged_data_cost.groupby(['channel_name', 'event_date'])['cost'].sum().reset_index()
channel_reporting_cost = channel_reporting_cost.rename(columns={'event_date': 'date'})

# --- Calculate channel_reporting_ihc ---
merged_data_ihc = pd.merge(attribution_customer_journey_df, session_sources_df, on='session_id')
channel_reporting_ihc = merged_data_ihc.groupby(['channel_name', 'event_date'])['ihc'].sum().reset_index()
channel_reporting_ihc = channel_reporting_ihc.rename(columns={'event_date': 'date'})

# --- Calculate channel_reporting_ihc_revenue ---
merged_conversions = pd.merge(conversions_df, attribution_customer_journey_df, on='conv_id')
merged_data_revenue = pd.merge(merged_conversions, session_sources_df, on='session_id')
merged_data_revenue['ihc_revenue'] = merged_data_revenue['ihc'] * merged_data_revenue['revenue']
channel_reporting_ihc_revenue = merged_data_revenue.groupby(['channel_name', 'event_date'])['ihc_revenue'].sum().reset_index()
channel_reporting_ihc_revenue = channel_reporting_ihc_revenue.rename(columns={'event_date': 'date'})

# --- Merge all KPIs into channel_reporting ---
channel_reporting = pd.merge(channel_reporting_cost, channel_reporting_ihc, on=['channel_name', 'date'], how='outer')
channel_reporting = pd.merge(channel_reporting, channel_reporting_ihc_revenue, on=['channel_name', 'date'], how='outer')

# Round the columns to 3 decimal places
channel_reporting['cost'] = channel_reporting['cost'].round(2)
channel_reporting['ihc'] = channel_reporting['ihc'].round(3)
channel_reporting['ihc_revenue'] = channel_reporting['ihc_revenue'].round(3)

In [7]:
channel_reporting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   channel_name  150 non-null    object 
 1   date          150 non-null    object 
 2   cost          150 non-null    float64
 3   ihc           117 non-null    float64
 4   ihc_revenue   117 non-null    float64
dtypes: float64(3), object(2)
memory usage: 6.0+ KB
